# Serving Estimator Models with Serverless Endpoints

## Retrieving and Attaching to a Completed Training Job


In [ ]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn

# Initialize SageMaker session
sagemaker_session = sagemaker.Session()

# List most recent completed training job
training_jobs = sagemaker_session.sagemaker_client.list_training_jobs(
    SortBy='CreationTime',                 # Sort jobs by creation time
    SortOrder='Descending',                # Newest jobs first
    StatusEquals='Completed',              # Only include completed jobs
    NameContains='sagemaker-scikit-learn'  # Filter to sklearn estimator jobs
)

# Extract the name of the latest training job
TRAINING_JOB_NAME = training_jobs['TrainingJobSummaries'][0]['TrainingJobName']

In [ ]:
# Attach to existing training job
estimator = SKLearn.attach(TRAINING_JOB_NAME)
print(f"Framework version: {estimator.framework_version}")
print(f"Model data S3 location: {estimator.model_data}")
print(f"Container image URI: {estimator.image_uri}")


## Configuring Serverless Inference


In [ ]:
from sagemaker.serverless import ServerlessInferenceConfig

# Configure serverless inference
serverless_config = ServerlessInferenceConfig(
    memory_size_in_mb=2048,  # Memory allocation
    max_concurrency=10       # Max concurrent requests
)

## Deploying the Estimator to a Serverless Endpoint


In [ ]:
# Configuration
ENDPOINT_NAME = "california-housing-estimator"

try:
    # Deploy as serverless endpoint asynchronously
    predictor = estimator.deploy(
        serverless_inference_config=serverless_config,
        endpoint_name=ENDPOINT_NAME,
        wait=False
    )

    # Retrieve detailed information about the specified SageMaker endpoint
    endpoint_description = sagemaker_session.sagemaker_client.describe_endpoint(EndpointName=ENDPOINT_NAME)

    # Extract the current status of the endpoint from the response
    status = endpoint_description['EndpointStatus']

    # Display the endpoint's status
    print(f"Endpoint status: {status}")
    
except Exception as e:
    print(f"Error: {e}")

## Testing the Deployed Endpoint for Inference


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
from sagemaker.predictor import Predictor

# Configuration
ENDPOINT_NAME = "california-housing-estimator-moein-2"
TEST_DATA_FILE = "data/california_housing_test.csv"

# Connect to the deployed SageMaker endpoint for inference
predictor = Predictor(endpoint_name=ENDPOINT_NAME)

# Set the serializer to convert input data (numpy arrays, pandas DataFrames) to CSV format for the endpoint
predictor.serializer = CSVSerializer()

# Set the deserializer to convert the CSV response from the endpoint back to Python objects (e.g., numpy arrays)
predictor.deserializer = CSVDeserializer()

In [ ]:
# Load and split test data into features and target variable
test_df = pd.read_csv(TEST_DATA_FILE)
X_test = test_df.drop("MedHouseVal", axis=1)
y_test = test_df["MedHouseVal"]

# Make predictions on entire test set
predictions = predictor.predict(X_test.values)

# Evaluate the model
test_r2 = r2_score(y_test, predictions)
test_rmse = np.sqrt(mean_squared_error(y_test, predictions))

# Print the results
print(f"R² Score: {test_r2:.4f}")
print(f"RMSE: {test_rmse:.4f}")